In [1]:
import os
import requests
import zipfile
import pandas as pd
import glacierml as gl
from PIL import Image
import numpy as np
from tqdm import tqdm
import glob
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

print(np.__version__)

1.23.5


In [2]:
RGI_zip_path = os.path.join(RGI_path,'RGI.zip')
RGI_download_link = 'https://www.glims.org/RGI/rgi60_files/00_rgi60_attribs.zip'

In [3]:
glathida_zip_path = os.path.join(glathida_path,'glathida.zip')
glathida_download_link = 'https://www.gtn-g.ch/database/glathida-3.1.0.zip'

In [4]:
ref_zip_path = os.path.join(ref_path,'')
ref_download_link = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/composite_thickness_RGI60-all_regions.zip?sequence=2&isAllowed=y'

In [5]:
def download_data(save_path, download_link, data):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print('data directory created')
    else:
        print('data directory exists')

    files = os.listdir(save_path)

    if not files:
        response = requests.get(download_link, stream=True)
        
        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))  # Total size in bytes
            
            print('downloading...')
            # Download with progress bar
            with open(os.path.join(save_path, data + '.zip'), 'wb') as file:
                with tqdm(total=total_size, unit='B', unit_scale=True, desc=data) as progress_bar:
                    for chunk in response.iter_content(1024):  # Download in 1 KB chunks
                        if chunk:  # Filter out keep-alive chunks
                            file.write(chunk)
                            progress_bar.update(len(chunk))  # Update progress bar
            print('download complete')

    else:
        zip_files = [file for file in files if file.endswith('.zip')]
        if zip_files:
            print('directory contains .zip files:', zip_files)


In [6]:
download_data(
    glathida_path,
    glathida_download_link,
    data = 'glathida'
)

data directory exists
directory contains .zip files: ['glathida.zip']


In [7]:
download_data(
    RGI_path,
    RGI_download_link,
    data = 'RGI'
)

data directory exists
directory contains .zip files: ['RGI.zip']


In [8]:
download_data(
    ref_path,
    ref_download_link,
    data = 'refs'
)

data directory exists
directory contains .zip files: ['refs.zip']


In [9]:
glathida_file = 'glathida-3.1.0/data/T.csv'
zip_path = os.path.join(glathida_path, 'glathida.zip')
destination_file = os.path.join(glathida_path, 'T.csv')

with zipfile.ZipFile(zip_path, 'r') as zipped_file:
    if os.path.exists(destination_file):
        print(f'data already unzipped to {glathida_path}')
    else:
        if glathida_file in zipped_file.namelist():
            with zipped_file.open(glathida_file) as source, open(destination_file, 'wb') as dest:
                # Get the total size of the file to unzip
                file_size = zipped_file.getinfo(glathida_file).file_size
                
                # Initialize the tqdm progress bar
                with tqdm(total=file_size, unit='B', unit_scale=True, desc='Unzipping') as progress_bar:
                    # Read in chunks and update the progress bar
                    for chunk in iter(lambda: source.read(1024), b''):  # Read in 1 KB chunks
                        dest.write(chunk)
                        progress_bar.update(len(chunk))  # Update the progress bar
            print(f'data unzipped to {glathida_path}')


data already unzipped to /home/simonhans/glacierml/data/glathida


In [10]:
zip_path = os.path.join(RGI_path, 'RGI.zip')
csv_files = glob.glob(os.path.join(RGI_path, '*.csv'))

with zipfile.ZipFile(zip_path, 'r') as zipped_file:
    if csv_files:
        print(f'data already unzipped to {RGI_path}')
    else:
        # Get the list of all files in the ZIP
        file_list = zipped_file.namelist()
        total_files = len(file_list)
        
        # Initialize progress bar for file extraction
        with tqdm(total=total_files, unit='file', desc='Unzipping files') as progress_bar:
            # Extract each file and update the progress bar
            for file in file_list:
                zipped_file.extract(file, RGI_path)
                progress_bar.update(1)  # Update the progress bar after extracting each file
        
        print(f'data unzipped to {RGI_path}')


data already unzipped to /home/simonhans/glacierml/data/RGI


In [11]:
zip_path = os.path.join(ref_path, 'refs.zip')

if os.path.exists(os.path.join(ref_path, 'refs.pkl')):
    print(f'data already unzipped to {ref_path}')
else:
    with zipfile.ZipFile(zip_path, 'r') as zipped_file:
        # Get the list of all files in the ZIP
        file_list = zipped_file.namelist()
        total_files = len(file_list)
        
        # Initialize progress bar for file extraction
        with tqdm(total=total_files, unit='file', desc='Unzipping files') as progress_bar:
            # Extract each file and update the progress bar
            for file in file_list:
                zipped_file.extract(file, ref_path)
                progress_bar.update(1)  # Update the progress bar after extracting each file
        
        print(f'data unzipped to {ref_path}')


Unzipping files: 100%|██████████████| 215563/215563 [00:49<00:00, 4345.04file/s]

data unzipped to /home/simonhans/glacierml/data/ref


In [12]:
RGIIds = []
means = []

if not os.path.exists(os.path.join(ref_path, 'refs.pkl')):
    print('Finding mean of Farinotti et al. 2019 estimates')

    # Get the list of RGI folders to process
    rgi_folders = [folder for folder in os.listdir(ref_path) if folder.startswith('RGI')]

    # Initialize a progress bar for the RGI folders
    for folder in tqdm(rgi_folders, desc="Processing RGI folders"):
        region_path = os.path.join(ref_path, folder)

        # Get the list of files in each RGI folder
        glac_files = os.listdir(region_path)

        # Initialize a progress bar for the files within the folder
        for file in tqdm(glac_files, desc=f"Processing files in {folder}", leave=False):
            glac_path = os.path.join(region_path, file)

            # Open and process each image
            im = Image.open(glac_path)
            arr = np.array(im)

            # Check if the array contains non-zero values
            if np.count_nonzero(arr) > 0:
                arr[arr == 0] = np.nan
                mean_thickness = np.nanmean(arr)
                means.append(mean_thickness)
                RGIIds.append(file[:-14])
            else:
                print(f"Array in file {file} is empty. Skipping...")

    # Create DataFrame and save it as a pickle file
    df = pd.concat(
        [
            pd.Series(RGIIds, name='RGIId'),
            pd.Series(means, name='FMT'),
        ], axis=1
    )
    df.to_pickle(os.path.join(ref_path, 'refs.pkl'))
    print('all done')

else:
    print('already done')


Finding mean of Farinotti et al. 2019 estimates


Processing files in RGI60-06:  60%|█████▉    | 338/568 [00:00<00:00, 467.46it/s]

Array in file RGI60-06.00560_thickness.tif is empty. Skipping...



Processing files in RGI60-19:   9%|▊        | 255/2751 [00:00<00:03, 651.39it/s]

Array in file RGI60-19.01323_thickness.tif is empty. Skipping...


...

Array in file RGI60-19.01425_thickness.tif is empty. Skipping...



Processing RGI folders:  53%|██████████         | 10/19 [01:51<00:50,  5.56s/it]

Array in file RGI60-19.01428_thickness.tif is empty. Skipping...



Processing RGI folders: 100%|███████████████████| 19/19 [05:04<00:00, 16.03s/it]


all done
